In [157]:
import pandas as pd
import numpy as np
import math
import hvplot.pandas
import ffn
from datetime import datetime
import yfinance as yf
%matplotlib inline

In [190]:
data = pd.read_csv('us_stock_data/us_stock_returns_cleaned.csv',index_col=0)
index = (data + 1).cumprod()

In [192]:
def mom_long_short(df, i, portfolios):
    mom_ret = np.log(df/df.shift(i))
    returns = np.log(df/df.shift(1))

    quintile_ranks = pd.DataFrame(index=mom_ret.index, columns=mom_ret.columns)
    for date in mom_ret.index:
        row_values = mom_ret.loc[date] # simply looping values
        ranks = pd.Series(row_values).dropna().rank(method='max') # set rank, exclude NaN values

        if not ranks.empty:  # Check if there are valid ranks
            quintiles = pd.qcut(ranks, q=portfolios, labels=False) # divide to quintiles
            quintile_ranks.loc[date, ranks.index] = quintiles # sets with quintiles they're in (5 is highest, 0 is lowest)

    quintile_dfs = {} # a dict of df's for each quintile (their returns)
    portfolio_returns = pd.DataFrame()
    for quintile in range(portfolios):
        filtered_df = returns[quintile_ranks == quintile]
        quintile_dfs[quintile] = filtered_df
        #For Equal Weight
        portfolio_returns[quintile] = quintile_dfs[quintile].mean(axis=1).dropna()

    cumulative_return = (1 + portfolio_returns).prod() - 1
    num_periods = len(df)
    annualized_return = (1 + cumulative_return) ** (12 / num_periods) - 1
    ann_vol = portfolio_returns.std()*math.sqrt(12)
    risk_adj_return = annualized_return/ann_vol

    plot = risk_adj_return.hvplot(kind='bar',title = 'Portfolios - Risk Adjusted Return', height = 300, width= 500, grid=True, color = 'teal')
    plot2 = (1+portfolio_returns).cumprod().hvplot(title = 'Portfolios - Log Scale', height = 300, width= 500, grid=True, logy=True)
    long_short = portfolio_returns[portfolios-1]-portfolio_returns[0]
    
    return portfolio_returns, plot, plot2, long_short

In [194]:
output = mom_long_short(index,60,8)
display(output[1])
display(output[2])

:Bars   [index]   (0)

:NdOverlay   [Variable]
   :Curve   [index]   (value)